In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
import sys, os

#sys.path.append(os.path.abspath(os.path.join("../..")))
sys.path.append(os.path.abspath(os.path.join("./scripts")))

In [3]:
import cleaning_functions as clnf
import plotting_functions as pltf

Import data as a dataframe

In [4]:
df = pd.read_csv("data/Week1_challenge_data_source.csv")

Before Cleaning the dataset, we explore what has been stored in it. The column names, how many data points, number of colum etc. All these questions needs to be answered

In [5]:
print("The data has {} rows and {} columns".format(df.shape[0], df.shape[1]))

The data has 150001 rows and 55 columns


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

In [7]:
fl_descp = pd.read_excel("data/Field_Descriptions.xlsx")
fl_descp.head(5)

,Fields,Description
0,bearer id,xDr session identifier
1,Dur. (ms),Total Duration of the xDR (in ms)
2,Start,Start time of the xDR (first frame timestamp)
3,Start ms,Milliseconds offset of start time for the xDR ...
4,End,End time of the xDR (last frame timestamp)


# Cleaning The Data

In [8]:
miss_df = clnf.missing_values_table(df)

Your selected dataframe has 55 columns.
There are 41 columns that have missing values.


In [9]:
miss_df

,Missing Values,% of Total Values,Dtype
Nb of sec with 37500B < Vol UL,130254,86.8,float64
Nb of sec with 6250B < Vol UL < 37500B,111843,74.6,float64
Nb of sec with 125000B < Vol DL,97538,65.0,float64
TCP UL Retrans. Vol (Bytes),96649,64.4,float64
Nb of sec with 31250B < Vol DL < 125000B,93586,62.4,float64
Nb of sec with 1250B < Vol UL < 6250B,92894,61.9,float64
Nb of sec with 6250B < Vol DL < 31250B,88317,58.9,float64
TCP DL Retrans. Vol (Bytes),88146,58.8,float64
HTTP UL (Bytes),81810,54.5,float64
HTTP DL (Bytes),81474,54.3,float64


We see that some columns are missing a lot of values. Handeling missing values depend on:
1. percentage of missing values
2. type of values
3. distribution of values

0. we drop columns missing > 50% of values

In [10]:
cols_to_drop = miss_df[miss_df['% of Total Values'] > 15].index.to_list()

In [11]:
df_clean = df.drop(cols_to_drop, axis = 1)

1. Columns that include unique identifiers, like 'Bearer Id', 'IMSI', 'IMEI'. Rows that have missing ids are useless and should be droped

In [12]:
fl_descp[fl_descp['Fields'].apply(lambda x: x in ['bearer id', 'IMSI', 'IMEI'])]

,Fields,Description
0,bearer id,xDr session identifier
7,IMSI,International Mobile Subscriber Identity
9,IMEI,International Mobile Equipment Identity


In [13]:
df_clean = df_clean.dropna(subset=['Bearer Id', 'IMSI', 'IMEI'])

# update what are the missing values in this new dataframe df_clean

miss_df = clnf.missing_values_table(df_clean)

Your selected dataframe has 43 columns.
There are 12 columns that have missing values.


2. "Last Location Name", "MSI SDN/Number" and "DL TP < 50 Kbps (%)" columns have little missing data, we can drop them

In [14]:
fl_descp[fl_descp['Fields'].apply(lambda x: x in ["Last Location Name", "MSISDN/Number"])]

,Fields,Description
8,MSISDN/Number,MS International PSTN/ISDN Number of mobile - ...
10,Last Location Name,User location call name (2G/3G/4G) at the end ...


In [15]:
fl_descp[fl_descp['Fields'] == 'DL TP < 50 Kbps (%)'].Description.to_list()

['Duration ratio when Bearer Downlink Throughput < ….']

In [16]:
df_clean = df_clean.dropna(subset=["Last Location Name", "MSISDN/Number"])

In [17]:
miss_df = clnf.missing_values_table(df_clean)

Your selected dataframe has 43 columns.
There are 10 columns that have missing values.


In [18]:
DL_columns = ['DL TP > 1 Mbps (%)', '250 Kbps < DL TP < 1 Mbps (%)', 
              '50 Kbps < DL TP < 250 Kbps (%)', 'DL TP < 50 Kbps (%)']

UL_columns = ['UL TP > 300 Kbps (%)', '50 Kbps < UL TP < 300 Kbps (%)', 
              '10 Kbps < UL TP < 50 Kbps (%)', 'UL TP < 10 Kbps (%)']

3. DL_columns and UL_columns have little missing data, we can drop them

In [19]:
fl_descp[fl_descp['Fields'].apply(lambda x: (x in DL_columns) | (x in UL_columns))]

,Fields,Description
17,DL TP < 50 Kbps (%),Duration ratio when Bearer Downlink Throughput...
18,50 Kbps < DL TP < 250 Kbps (%),Duration ratio when Bearer Downlink Throughput...
19,250 Kbps < DL TP < 1 Mbps (%),Duration ratio when Bearer Downlink Throughput...
20,DL TP > 1 Mbps (%),Duration ratio when Bearer Downlink Throughput...
21,UL TP < 10 Kbps (%),Duration ratio when Bearer Uplink Throughput < ….
22,10 Kbps < UL TP < 50 Kbps (%),Duration ratio when Bearer Uplink Throughput r...
23,50 Kbps < UL TP < 300 Kbps (%),Duration ratio when Bearer Uplink Throughput r...
24,UL TP > 300 Kbps (%),Duration ratio when Bearer Uplink Throughput > ….


In [20]:
df_clean= df_clean.dropna(how = 'all', subset= DL_columns)
df_clean= df_clean.dropna(how = 'all', subset= UL_columns)

miss_df = clnf.missing_values_table(df_clean)

Your selected dataframe has 43 columns.
There are 2 columns that have missing values.


In [21]:
df_clean= df_clean.dropna(subset= ['Nb of sec with Vol DL < 6250B', 'Nb of sec with Vol UL < 1250B'])

miss_df = clnf.missing_values_table(df_clean)
miss_df

Your selected dataframe has 43 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values,Dtype


Finally we change the data type in columns 'Start' and 'End' to datetime

In [22]:
df_clean['Start']= pd.to_datetime(df_clean['Start'])

In [23]:
df_clean['End']= pd.to_datetime(df_clean['End'])

In [24]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146891 entries, 0 to 149999
Data columns (total 43 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   Bearer Id                       146891 non-null  float64       
 1   Start                           146891 non-null  datetime64[ns]
 2   Start ms                        146891 non-null  float64       
 3   End                             146891 non-null  datetime64[ns]
 4   End ms                          146891 non-null  float64       
 5   Dur. (ms)                       146891 non-null  float64       
 6   IMSI                            146891 non-null  float64       
 7   MSISDN/Number                   146891 non-null  float64       
 8   IMEI                            146891 non-null  float64       
 9   Last Location Name              146891 non-null  object        
 10  Avg Bearer TP DL (kbps)         146891 non-null  float64

# Exploring The Data

## Univariate Summary Statistics

In [25]:
identity_cols = ['MSISDN/Number','IMSI', 'IMEI', 'Bearer Id']

In [26]:
df_clean.drop(identity_cols, axis=1).describe()

,Start ms,End ms,Dur. (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),DL TP < 50 Kbps (%),50 Kbps < DL TP < 250 Kbps (%),250 Kbps < DL TP < 1 Mbps (%),DL TP > 1 Mbps (%),UL TP < 10 Kbps (%),...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
count,146891.000000,146891.000000,1.468910e+05,146891.000000,146891.000000,146891.000000,146891.000000,146891.000000,146891.000000,146891.000000,...,1.468910e+05,1.468910e+05,1.468910e+05,1.468910e+05,1.468910e+05,1.468910e+05,1.468910e+05,1.468910e+05,1.468910e+05,1.468910e+05
mean,499.117189,498.654397,1.052299e+05,13419.455991,1788.190618,92.837893,3.064762,1.721263,1.618547,98.528290,...,1.163341e+07,1.100858e+07,1.162650e+07,1.100286e+07,4.220649e+08,8.290257e+06,4.210038e+08,8.262887e+06,4.112139e+07,4.546628e+08
std,288.554839,288.073522,8.092446e+04,24081.550773,4650.560582,13.054204,6.212684,4.161456,4.838657,4.643045,...,6.710450e+06,6.345243e+06,6.724904e+06,6.357359e+06,2.440381e+08,4.781553e+06,2.431448e+08,4.768594e+06,1.127775e+07,2.442080e+08
min,0.000000,0.000000,7.142000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.300000e+01,1.050000e+02,4.200000e+01,3.500000e+01,2.516000e+03,5.900000e+01,3.290000e+03,1.480000e+02,2.866892e+06,7.114041e+06
25%,250.000000,251.000000,5.852700e+04,43.000000,47.000000,91.000000,0.000000,0.000000,0.000000,99.000000,...,5.833512e+06,5.516549e+06,5.776626e+06,5.480202e+06,2.103941e+08,4.132622e+06,2.101527e+08,4.144489e+06,3.321835e+07,2.430767e+08
50%,499.000000,500.000000,8.639900e+04,64.000000,63.000000,100.000000,0.000000,0.000000,0.000000,100.000000,...,1.161638e+07,1.101329e+07,1.164271e+07,1.099939e+07,4.234929e+08,8.294516e+06,4.216332e+08,8.265817e+06,4.114253e+07,4.559702e+08
75%,749.000000,750.000000,1.331545e+05,20023.500000,1136.000000,100.000000,4.000000,1.000000,0.000000,100.000000,...,1.744064e+07,1.651411e+07,1.747017e+07,1.650352e+07,6.333301e+08,1.243239e+07,6.313683e+08,1.238204e+07,4.903488e+07,6.657831e+08
max,999.000000,999.000000,1.859336e+06,378160.000000,58613.000000,100.000000,93.000000,76.000000,94.000000,100.000000,...,2.325910e+07,2.201196e+07,2.325919e+07,2.201196e+07,8.434419e+08,1.655879e+07,8.434425e+08,1.655882e+07,7.833131e+07,9.029696e+08


In [27]:
df_clean[['Handset Type', 'Handset Manufacturer', 'Last Location Name']].describe()

,Handset Type,Handset Manufacturer,Last Location Name
count,146891,146891,146891
unique,1363,168,45084
top,Huawei B528S-23A,Apple,D41377B
freq,19373,59072,80


In [28]:
# 5 Top Handset Manufacturers

df_clean['Handset Manufacturer'].value_counts().head(5)

Apple                            59072
Samsung                          39902
Huawei                           33755
undefined                         8820
Sony Mobile Communications Ab      925
Name: Handset Manufacturer, dtype: int64

In [29]:
# 5 Top Handset Types

df_clean['Handset Type'].value_counts().head(5)

Huawei B528S-23A           19373
Apple iPhone 6S (A1688)     9369
Apple iPhone 6 (A1586)      8967
undefined                   8820
Apple iPhone 7 (A1778)      6240
Name: Handset Type, dtype: int64

## Aggregate Data Per User

We choose to identify user by their phone numbers (MSISDN/Number)

### 1. number of xDR sessions per user

In [30]:
num_sessions_user = clnf.find_agg(df_clean, 'MSISDN/Number', 'count', 'Bearer Id')

In [31]:
num_sessions_user.head(5)

,MSISDN/Number,Bearer Id
13381,3.362632e+10,18
6353,3.361489e+10,17
13037,3.362578e+10,17
36669,3.365973e+10,16
75564,3.367588e+10,15


In [32]:
df_clean['MSISDN/Number'].nunique()

105719

### 2. Session duration Per user

In [36]:
session_dur_user = clnf.find_agg(df_clean, 'MSISDN/Number', 'sum', 'Dur. (ms)')
session_dur_user.head(5)

,MSISDN/Number,Dur. (ms)
13037,3.362578e+10,18553754.0
6353,3.361489e+10,9966898.0
91949,3.376054e+10,9279434.0
13381,3.362632e+10,8791927.0
64472,3.366716e+10,8744914.0


### 3. the total download (DL) and upload (UL) data Per User

In [38]:
total_download_user = df_clean.groupby('MSISDN/Number')[['Total DL (Bytes)', 'Total UL (Bytes)']].sum()
total_download_user.head(5)

,Total DL (Bytes),Total UL (Bytes)
MSISDN/Number,,
3.360100e+10,8.426375e+08,36053108.0
3.360100e+10,1.207552e+08,36104459.0
3.360100e+10,5.566597e+08,39306820.0
3.360101e+10,4.019932e+08,20327526.0
3.360101e+10,1.363130e+09,94280527.0


### 4. the total data volume (in Bytes) during this session for each application

In [43]:
data_per_application = df_clean.groupby(['MSISDN/Number','Bearer Id'])[['Social Media DL (Bytes)',
       'Social Media UL (Bytes)', 'Google DL (Bytes)', 'Google UL (Bytes)',
       'Email DL (Bytes)', 'Email UL (Bytes)', 'Youtube DL (Bytes)',
       'Youtube UL (Bytes)', 'Netflix DL (Bytes)', 'Netflix UL (Bytes)',
       'Gaming DL (Bytes)', 'Gaming UL (Bytes)', 'Other DL (Bytes)',
       'Other UL (Bytes)']].sum()

data_per_application.head(5)

Social Media DL (Bytes)  Social Media UL (Bytes)  \
MSISDN/Number Bearer Id                                                        
3.360100e+10  1.311448e+19                2206504.0                  25631.0   
              7.349883e+18                2598548.0                  62017.0   
              7.349883e+18                3148004.0                  47619.0   
3.360101e+10  1.304243e+19                 251469.0                  28825.0   
              7.277826e+18                1501326.0                  22731.0   

                            Google DL (Bytes)  Google UL (Bytes)  \
MSISDN/Number Bearer Id                                            
3.360100e+10  1.311448e+19          3337123.0          1051882.0   
              7.349883e+18          4197697.0          1137166.0   
              7.349883e+18          3343483.0            99643.0   
3.360101e+10  1.304243e+19          5937765.0          3740728.0   
              7.277826e+18          7419606.0          2751029.0   

                            Email DL (Bytes)  Email UL (Bytes)  \
MSISDN/Number Bearer Id                                          
3.360100e+10  1.311448e+19          837400.0          493962.0   
              7.349883e+18         2828821.0          478960.0   
              7.349883e+18         2436500.0          768880.0   
3.360101e+10  1.304243e+19         2178618.0          106052.0   
              7.277826e+18         1267949.0          145736.0   

                            Youtube DL (Bytes)  Youtube UL (Bytes)  \
MSISDN/Number Bearer Id                                              
3.360100e+10  1.311448e+19          14900201.0           6724347.0   
              7.349883e+18           5324251.0           7107972.0   
              7.349883e+18           2137272.0          19196298.0   
3.360101e+10  1.304243e+19           4393123.0           2584198.0   
              7.277826e+18           2312338.0          18180144.0   

                            Netflix DL (Bytes)  Netflix UL (Bytes)  \
MSISDN/Number Bearer Id                                              
3.360100e+10  1.311448e+19          10265105.0          16915876.0   
              7.349883e+18            770569.0          10451194.0   
              7.349883e+18          16525919.0           2827981.0   
3.360101e+10  1.304243e+19           1157362.0            784730.0   
              7.277826e+18           3794492.0          18178618.0   

                            Gaming DL (Bytes)  Gaming UL (Bytes)  \
MSISDN/Number Bearer Id                                            
3.360100e+10  1.311448e+19        811091133.0          1367528.0   
              7.349883e+18        105035298.0         14714780.0   
              7.349883e+18        529068485.0          9759228.0   
3.360101e+10  1.304243e+19        388074835.0          3051292.0   
              7.277826e+18        799737769.0          2827420.0   

                            Other DL (Bytes)  Other UL (Bytes)  
MSISDN/Number Bearer Id                                         
3.360100e+10  1.311448e+19       377096990.0         9473882.0  
              7.349883e+18       279557701.0         2152370.0  
              7.349883e+18       495086501.0         6607171.0  
3.360101e+10  1.304243e+19        25248001.0        10031701.0  
              7.277826e+18        72295238.0        13693422.0